In [0]:
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm
import ast
import requests
import apikey
import os
from pydantic import BaseModel
from typing import Deque, List, Optional, Tuple
from ast import literal_eval


In [0]:
pswd = apikey.load("Perplexity API key", filename=r'path to your key')

In [0]:
###get a list of the websites you want loop through Perplexity
df=pd.read_csv('path to /YOUR LIST OF URLs.csv')
urls=df.Website.dropna().values.tolist()

In [0]:
df_all=pd.DataFrame() ### table to add results of each API/website call

In [0]:
class AnswerFormat(BaseModel):
    company_name: str
    location: str
    year_of_incorporation: str 
    number_of_employees: int
    amount_money_raised: str 
    core_technology: str 
    core_products: str 
    website_summary: str
    use_cases: str


for link in tqdm(urls):
    perplexity_url = "https://api.perplexity.ai/chat/completions"
    # "model": "sonar-reasoning-pro",
    headers = {"Authorization": f"Bearer {pswd}"}
    payload = {
        "model": "sonar",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content":
                f"extract the information from this website: {link} and from the Internet about this company and return the information about company name, use cases they target, location of the main office, year of incorporation, total amount of investments raised, list of core technologies, list of core products, and website summary "
             "Please output a JSON object containing the following fields: company_name, use_cases, location, year_of_incorporation, number_of_employees,amount_money_raised, core_technology, core_products, website_summary. "
            }
        ],
        
        "response_format": {
    		    "type": "json_schema",
            "json_schema": {"schema": AnswerFormat.model_json_schema()},
        },
        # "search_domain_filter": [
        #     f"{link}",
        #     #"wikipedia.org",
        #     "crunchbase.com",
        #     "cbinsights.com",
        #     "pitchbook.com"
        # ]
    }
    
    response = requests.post(perplexity_url, headers=headers, json=payload).json()
    res_str=response["choices"][0]["message"]["content"]
    df_all=pd.concat([df_all, pd.DataFrame.from_dict(literal_eval(res_str), orient='index').T])

100%|██████████| 157/157 [12:35<00:00,  4.81s/it]


In [0]:
df_all.to_csv('path to/NAME YOUR OUTPUT FILE.csv')

In [0]:
df_all